# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [ ]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [1]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


o5bIcLwjlqD1zhLOd4jPC0nu8ulknRgJMigWt0mD


In [2]:
## bloque conexion a Cohere
import cohere
co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)



c:\Users\marti\OneDrive\Escritorio\challenge_prompting_notekook\venv\Lib\site-packages\cohere\core\pydantic_utilities.py:13: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.datetime_parse import parse_date as parse_date


id='e92c903f-9fdc-4fb8-b57d-2310250bd9b5' finish_reason='COMPLETE' message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0), cached_tokens=192.0) logprobs=None


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [ ]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

In [ ]:
#####

In [10]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

# Texto 1 - Formato narrativo desordenado
text_to_analize_1 = """Tras sufrir una caída en su domicilio, el señor Roberto Jiménez, quien cuenta con setenta y dos años de edad, 
fue trasladado de urgencia al Centro de Traumatología. Presentaba dolor intenso en la cadera izquierda. 
La fecha de ingreso fue 14-Feb-2024. Radiografías confirmaron fractura de fémur. 
El tratamiento indicado por el Dr. Mendoza incluye cirugía programada y analgesia. 
Su próxima valoración será dentro de 15 días."""

# Texto 2 - Formato telegráfico con datos mezclados
text_to_analize_2 = """INGRESO: 2023/12/08 - Paciente: Laura, 19 años. 
Motivo consulta: cefalea intensa + fotofobia + náuseas. 
Establecido diagnóstico: migraña con aura. 
Médico tratante: Dra. Silva. 
Plan: triptanes + reposo en habitación oscura. 
Próximo control: 20 de diciembre 2023."""

# Texto 3 - Formato con datos al final y edad en palabras
text_to_analize_3 = """Acude a consulta externa por presentar lesiones en mucosa oral y dificultad para tragar. 
Se realiza biopsia que confirma diagnóstico de estomatitis aftosa severa. 
Se pauta tratamiento con enjuagues bucales medicados y corticoides tópicos. 
La paciente se llama Carmen Ríos, tiene cuarenta y siete años. 
Fecha de la consulta: marzo 5, 2024. 
Seguimiento en un mes."""

# Texto 4 - Formato con datos entremezclados y fecha en texto completo
text_to_analize_4 = """En el día de hoy, diez de agosto del dos mil veintitrés, 
recibimos al niño Miguel Ángel Santos, de 8 años de edad, 
por cuadro de fiebre de 39°C y erupción cutánea maculopapular. 
Confirmado diagnóstico de varicela mediante test específico. 
La pediatra Daniela Ortiz recomienda aislamiento y antipiréticos. 
Próxima revisión: cuando desaparezcan las lesiones."""

# Texto 5 - Formato muy desestructurado con datos incompletos
text_to_analize_5 = """Paciente femenina, nombre: Patricia Navarro. Edad: treinta y un años. 
Síntomas referidos: dolor abdominal en hipogastrio, disuria. 
Examen de orina positivo para infección. 
Diagnóstico: cistitis aguda. 
Atendida en Sala de Emergencias el 30/06/2024. 
Tratamiento instaurado: antibiótico (nitrofurantoína) y aumento de ingesta hídrica. 
Volver a consultar si persisten síntomas."""

In [25]:
# your code here
import cohere
co = cohere.ClientV2()
response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{
                "role": "system", 
                "content": """Eres un asistente que extrae información médica relevante de textos clínicos y la estructura en formato JSON.
                
                Instrucciones:
                - Extrae solo la información clave del texto proporcionado
                - Preséntala en formato JSON estructurado
                - No inventes información que no esté en el texto
                - Si no hay información disponible para algún campo, déjalo vacío
                - La edad solo debe ser un número entero, por ejemplo "edad": 45
                - El formato de fecha debe ser "YYYY-MM-DD", por ejemplo "2023-08-05"
                
                Formato requerido:
                {
                    "paciente": {
                        "nombre": "",
                        "edad": 
                    },
                    "fecha_admision": "",
                    "sintomas": [],
                    "diagnostico": "",
                    "tratamiento": []
                }"""
            },
                {"role": "user", "content": f"Por favor, extrae la información médica del siguiente texto: {text_to_analize}"}],
    )

json_response = response.message.content[0].text

print(json_response)

response_1 = co.chat(
    model="command-r-plus-08-2024",
    messages=[{
                "role": "system", 
                "content": """Eres un asistente que extrae información médica relevante de textos clínicos y la estructura en formato JSON.
                
                Instrucciones:
                - Extrae solo la información clave del texto proporcionado
                - Preséntala en formato JSON estructurado
                - No inventes información que no esté en el texto
                - Si no hay información disponible para algún campo, déjalo vacío
                - La edad solo debe ser un número entero, por ejemplo "edad": 45
                - El formato de fecha debe ser "YYYY-MM-DD", por ejemplo "2023-08-05"
                
                Formato requerido:
                {
                    "paciente": {
                        "nombre": "",
                        "edad": 
                    },
                    "fecha_admision": "",
                    "sintomas": [],
                    "diagnostico": "",
                    "tratamiento": []
                }"""
            },
                {"role": "user", "content": f"Por favor, extrae la información médica del siguiente texto: {text_to_analize_1}"}],
    )

json_response_1 = response_1.message.content[0].text

print(json_response_1)

response_2 = co.chat(
    model="command-r-plus-08-2024",
    messages=[{
                "role": "system", 
                "content": """Eres un asistente que extrae información médica relevante de textos clínicos y la estructura en formato JSON.
                
                Instrucciones:
                - Extrae solo la información clave del texto proporcionado
                - Preséntala en formato JSON estructurado
                - No inventes información que no esté en el texto
                - Si no hay información disponible para algún campo, déjalo vacío
                - La edad solo debe ser un número entero, por ejemplo "edad": 45
                - El formato de fecha debe ser "YYYY-MM-DD", por ejemplo "2023-08-05"
                
                Formato requerido:
                {
                    "paciente": {
                        "nombre": "",
                        "edad": 
                    },
                    "fecha_admision": "",
                    "sintomas": [],
                    "diagnostico": "",
                    "tratamiento": []
                }"""
            },
                {"role": "user", "content": f"Por favor, extrae la información médica del siguiente texto: {text_to_analize_2}"}],
    )

json_response_2 = response_2.message.content[0].text

print(json_response_2)

response_3 = co.chat(
    model="command-r-plus-08-2024",
    messages=[{
                "role": "system", 
                "content": """Eres un asistente que extrae información médica relevante de textos clínicos y la estructura en formato JSON.
                
                Instrucciones:
                - Extrae solo la información clave del texto proporcionado
                - Preséntala en formato JSON estructurado
                - No inventes información que no esté en el texto
                - Si no hay información disponible para algún campo, déjalo vacío
                - La edad solo debe ser un número entero, por ejemplo "edad": 45
                - El formato de fecha debe ser "YYYY-MM-DD", por ejemplo "2023-08-05"
                
                Formato requerido:
                {
                    "paciente": {
                        "nombre": "",
                        "edad": 
                    },
                    "fecha_admision": "",
                    "sintomas": [],
                    "diagnostico": "",
                    "tratamiento": []
                }"""
            },
                {"role": "user", "content": f"Por favor, extrae la información médica del siguiente texto: {text_to_analize_3}"}],
    )

json_response_3 = response_3.message.content[0].text

print(json_response_3)

response_4 = co.chat(
    model="command-r-plus-08-2024",
    messages=[{
                "role": "system", 
                "content": """Eres un asistente que extrae información médica relevante de textos clínicos y la estructura en formato JSON.
                
                Instrucciones:
                - Extrae solo la información clave del texto proporcionado
                - Preséntala en formato JSON estructurado
                - No inventes información que no esté en el texto
                - Si no hay información disponible para algún campo, déjalo vacío
                - La edad solo debe ser un número entero, por ejemplo "edad": 45
                - El formato de fecha debe ser "YYYY-MM-DD", por ejemplo "2023-08-05"
                
                Formato requerido:
                {
                    "paciente": {
                        "nombre": "",
                        "edad": 
                    },
                    "fecha_admision": "",
                    "sintomas": [],
                    "diagnostico": "",
                    "tratamiento": []
                }"""
            },
                {"role": "user", "content": f"Por favor, extrae la información médica del siguiente texto: {text_to_analize_4}"}],
    )

json_response_4 = response_4.message.content[0].text

print(json_response_4)

response_5 = co.chat(
    model="command-r-plus-08-2024",
    messages=[{
                "role": "system", 
                "content": """Eres un asistente que extrae información médica relevante de textos clínicos y la estructura en formato JSON.
                
                Instrucciones:
                - Extrae solo la información clave del texto proporcionado
                - Preséntala en formato JSON estructurado
                - No inventes información que no esté en el texto
                - Si no hay información disponible para algún campo, déjalo vacío
                - La edad solo debe ser un número entero, por ejemplo "edad": 45
                - El formato de fecha debe ser "YYYY-MM-DD", por ejemplo "2023-08-05"
                
                Formato requerido:
                {
                    "paciente": {
                        "nombre": "",
                        "edad": 
                    },
                    "fecha_admision": "",
                    "sintomas": [],
                    "diagnostico": "",
                    "tratamiento": []
                }"""
            },
                {"role": "user", "content": f"Por favor, extrae la información médica del siguiente texto: {text_to_analize_5}"}],
    )

json_response_5 = response_5.message.content[0].text

print(json_response_5)


{
    "paciente": {
        "nombre": "Sofía López",
        "edad": 28
    },
    "fecha_admision": "2023-04-03",
    "sintomas": [
        "fiebre alta",
        "tos persistente"
    ],
    "diagnostico": "neumonía",
    "tratamiento": [
        "antibióticos",
        "reposo"
    ]
}
{
    "paciente": {
        "nombre": "Roberto Jiménez",
        "edad": 72
    },
    "fecha_admision": "2024-02-14",
    "sintomas": [
        "Dolor intenso en la cadera izquierda"
    ],
    "diagnostico": "Fractura de fémur",
    "tratamiento": [
        "Cirugía programada",
        "Analgesia"
    ]
}
{
    "paciente": {
        "nombre": "Laura",
        "edad": 19
    },
    "fecha_admision": "2023-12-08",
    "sintomas": [
        "cefalea intensa",
        "fotofobia",
        "náuseas"
    ],
    "diagnostico": "migraña con aura",
    "tratamiento": [
        "triptanes",
        "reposo en habitación oscura"
    ]
}
{
    "paciente": {
        "nombre": "Carmen Ríos",
        "edad": 47
 

In [29]:
# test

# final_result = json.loads(llm_response)
import cohere
import json

# Diccionario con todas las respuestas
respuestas = {
    "Texto Original": json_response,
    "Texto 1": json_response_1,
    "Texto 2": json_response_2,
    "Texto 3": json_response_3, 
    "Texto 4": json_response_4,
    "Texto 5": json_response_5
}

print("TESTS AUTOMÁTICOS")
print("=" * 50)

exitosos = 0
totales = len(respuestas)

for nombre, respuesta in respuestas.items():
    try:
        # Limpiar respuesta
        respuesta_limpia = respuesta
        if "```json" in respuesta_limpia:
            respuesta_limpia = respuesta_limpia.split("```json")[1].split("```")[0].strip()
        elif "```" in respuesta_limpia:
            respuesta_limpia = respuesta_limpia.split("```")[1].split("```")[0].strip()
        
        # Test
        final_result = json.loads(respuesta_limpia)
        print(f"{nombre}: EXITOSO")
        exitosos += 1
        
    except Exception as e:
        print(f"{nombre}: FALLIDO - {e}")

print("=" * 50)
print(f"RESULTADO: {exitosos}/{totales} tests exitosos")
print(f"EFECTIVIDAD: {(exitosos/totales)*100:.1f}%")

TESTS AUTOMÁTICOS
Texto Original: EXITOSO
Texto 1: EXITOSO
Texto 2: EXITOSO
Texto 3: EXITOSO
Texto 4: EXITOSO
Texto 5: EXITOSO
RESULTADO: 6/6 tests exitosos
EFECTIVIDAD: 100.0%


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [25]:
contacts = {
    'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
    'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

def add_contact(name, tel, mail):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'tel': tel, 'mail': mail}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."

In [2]:
print(get_information("Juan Pérez"))

print(add_contact("Juan Pérez", "555-1234", "juanperez@mail.com"))
print(contacts["Juan Pérez"]) 

Contacto no encontrado.
Contacto añadido con éxito.
{'tel': '555-1234', 'mail': 'juanperez@mail.com'}


In [48]:
# Creamos las tools para el LLM (https://docs.cohere.com/docs/tool-use-overview#system-message)
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Agrega un contacto con nombre, teléfono y email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto"
                    },
                    "tel": {
                        "type": "string",
                        "description": "Número de teléfono"
                    },
                    "mail": {
                        "type": "string",
                        "description": "Correo electrónico"
                    }
                },
                "required": ["name", "tel", "mail"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Obtiene la información de un contacto por nombre.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Nombre del contacto"
                    }
                },
                "required": ["name"]
            }
        }
    }
]


# Función para consultar al LLM

def consultar_llm(content_user):
    response = co.chat(
        model="command-r-plus-08-2024",
        tools=tools,
        tool_choice="auto",
        messages=[
            {
                "role": "system",
                "content": """Tu tarea es interpretar las consultas del usuario y decidir, si corresponde, llamar alguna de las funciones disponibles mediante tool calling. No inventes datos, no completes información faltante y no agregues contactos con información inexistente.
                Actuá como un agente que administra contactos personales. Respondé de forma breve, clara y siempre en español.  
                El usuario puede:
                - Agregar un contacto.
                - Consultar la información de un contacto.
                No inventes teléfonos, correos ni nombres.  
                No generes información sensible ni datos privados que el usuario no haya proporcionado explícitamente.  
                Si el usuario pide información que no existe en la agenda actual, respondé que no se encuentra o utilizá la función correspondiente para confirmarlo.  
                No supongas datos incompletos.
                Usá exclusivamente la información provista por el usuario y las funciones disponibles.  
                No uses conocimientos fuera del estado actual del diccionario de contactos.  
                Si el usuario no brinda suficiente información para llamar a una función, pedí la información faltante.
                La agenda de contactos es la siguiente: """ + str(contacts) + """

                Formatos aceptados:
                - Para agregar: name, tel, mail
                - Para consultar: name

                Interpretá cada consulta del usuario:  
                1. Si es una acción sobre contactos, llamá a la función adecuada usando tool calling.  
                2. Si no corresponde llamar a ninguna función, respondé con un mensaje breve y claro.  

                No inventes información bajo ninguna circunstancia.

                Responde usando exactamente este formato JSON (la respuesta debe ser generada como una cadena de texto, no agregues explicaciones de lo que vas a hacer para obtener la respuesta):
                { "respuesta": "", "citas_del_contexto": :p }
                """
            },
            {
                "role": "user",
                "content": content_user
            }
        ]
    )

        # Ejecutar el tool call para obtener datos
    if response.message.tool_calls:
        for call in response.message.tool_calls:
            if call.function.name == "get_information":
                args = json.loads(call.function.arguments)
                contact_data = get_information(args['name'])
                
                # Segunda llamada - generar respuesta formateada
                response2 = co.chat(
                    model="command-r-plus-08-2024",
                    messages=[
                        {
                            "role": "system",
                            "content": f"""Genera una respuesta directa en español usando este formato:
                            - Si preguntan por teléfono: "El teléfono de [nombre] es [tel]"
                            - Si preguntan por email: "El email de [nombre] es [mail]"
                            - Si no existe: "El contacto [nombre] no existe"
                            
                            Datos obtenidos: {contact_data}
                            Consulta del usuario: {content_user}"""
                        },
                        {
                            "role": "user",
                            "content": content_user
                        }
                    ]
                )
                return response2
    return response


In [49]:
# Funcion para procesar la respuesta del LLM y decidir a que funcion llamar
# Tenemos 2 casos para que funcione con diferentes modelos de LLM de cohere

import json

def procesar_respuesta(response):
    message = response.message

    if not message.tool_calls:
        return message.content[0].text

    for call in message.tool_calls:

        # Caso 1 — formato con function.name / function.arguments
        if hasattr(call, "function") and call.function is not None:
            name = call.function.name
            args = json.loads(call.function.arguments)

        # Caso 2 — formato con name / parameters directo
        else:
            name = call.name
            args = call.parameters


        # ejecutar la función correspondiente
        if name == "add_contact":
            return add_contact(**args)
        elif name == "get_information":
            return get_information(**args)

    return None


In [52]:
user_input = "Cuál es el correo electronico de Mariano gomez?"
r = consultar_llm(user_input)
print(r)
salida = procesar_respuesta(r)
print(salida)

print(contacts)

# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

id='e5dfcaf8-72e1-469f-a7fb-eece9e29106c' finish_reason='COMPLETE' message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='El email de Mariano gomez es mariano@example.com.')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=123.0, output_tokens=14.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=282.0, output_tokens=14.0), cached_tokens=144.0) logprobs=None
El email de Mariano gomez es mariano@example.com.
{'Joaquin Lopez': {'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}, 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}, 'Maria Gomez': {'tel': '555-6789', 'mail': 'maria@example.com'}, 'Mariano Gomez': {'tel': '5552-6789', 'mail': 'mariano@example.com'}}


In [51]:
user_input = "Agrega a Mariano Gomez, teléfono 5552-6789, email mariano@example.com"
r = consultar_llm(user_input)
salida = procesar_respuesta(r)
print(salida)

Contacto añadido con éxito.


## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [11]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

# 1. Detectar el idioma de la pregunta

import cohere
co = cohere.ClientV2()
def detectar_idioma(pregunta):
    prompt = f"Detecta el idioma de la siguiente pregunta: {pregunta} y responde solo con el nombre del idioma ('Español', 'Inglés', 'Portugués')."
    response = co.chat(
        model="command-a-vision-07-2025",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.message.content[0].text

In [12]:
print(detectar_idioma("¿Cómo estás?"))  # Debería devolver 'Español'
print(detectar_idioma("How are you?"))  # Debería devolver 'Inglés'
print(detectar_idioma("Como você está?"))  # Debería devolver 'Portugués'

Español
Inglés
Portugués


In [13]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""


# Preguntas relacionadas
pregunta1 = "¿Cómo cambió Thomas después de la batalla?"
pregunta2 = "How did Thomas change after the battle?"
pregunta3 = "Como Thomas mudou depois da batalha?"

# Preguntas NO relacionadas
pregunta4 = "¿Cuál es la capital de Francia?"
pregunta5 = "What is the capital of France?"


# respuesta
#print(history_answer(pregunta1))

In [35]:
def verificar_relacion(historia, pregunta):
    prompt = f"""Determina si la siguiente pregunta está relacionada con la historia proporcionada. 
    Responde solo con 'Sí.' o 'No.'.

    Historia: {historia}

    Pregunta: {pregunta}
    """
    response = co.chat(
        model="command-a-vision-07-2025",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.message.content[0].text

In [36]:
print(verificar_relacion(historia, pregunta1))  # Debería devolver 'Sí'
print(verificar_relacion(historia, pregunta2))  # Debería devolver 'Sí
print(verificar_relacion(historia, pregunta3))  # Debería devolver 'Sí'
print(verificar_relacion(historia, pregunta4))  # Debería devolver 'No'
print(verificar_relacion(historia, pregunta5))  # Debería devolver 'No'

Sí.
Sí.
Sí.
No.
No.


In [73]:
system_content = """Eres un asistente que debe responder preguntas basándose exclusivamente en el contexto.
                    Reglas obligatorias:
                    1. Responde en EXACTAMENTE UNA oración (no dos, no tres, solo UNA)
                    2. Escribe SIEMPRE en tercera persona (él, ella, Thomas, nunca "yo" o "tú")
                    3. Usa el mismo idioma que la pregunta
                    4. Incluye emojis relevantes y que tengan que ver con la respuesta dentro de la oración(SIN EXCEPCION).
                    5. SIEMPRE termina con "Hakuna Matata!" (en todas las respuestas sin excepción)
                    6. Basa tu respuesta solo en la historia proporcionada"""

import time

# Diccionario para cachear respuestas
cache_respuestas = {}

def generar_respuesta_final(historia, pregunta, idioma):
    # Crear una clave única para la pregunta
    clave_cache = pregunta.lower().strip()
    
    # Si ya respondimos esta pregunta, devolver la respuesta cacheada
    if clave_cache in cache_respuestas:
        return cache_respuestas[clave_cache]
    
    prompt = f"""CONTEXTO:
                {historia}

                PREGUNTA:
                {pregunta}

                Responde en {idioma}, en UNA SOLA ORACIÓN, en tercera persona, con emojis(obligatorios), y agrega siempre al final de la oración (sin excepción) lo siguiente: "Hakuna Matata!"."""
    
    response = co.chat(
        model="command-a-vision-07-2025",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2  # Temperatura 0 para máxima consistencia
    )
    
    respuesta = response.message.content[0].text
    
    # Guardar en caché
    cache_respuestas[clave_cache] = respuesta
    
    return respuesta

In [74]:
def history_answer(pregunta):
    # Detectar idioma
    idioma = detectar_idioma(pregunta)
    
    # Verificar relación
    relacion = verificar_relacion(historia, pregunta)
    
    # Comparar (maneja "Sí", "Sí.", "Si", etc.)
    if "Sí" in relacion or "Si" in relacion:
        respuesta_al_usuario = generar_respuesta_final(historia, pregunta, idioma)
    else:
        # Responder en el idioma correcto según la pregunta
        if "Inglés" in idioma:
            respuesta_al_usuario = "Sorry, I can't help you with that. Hakuna Matata!"
        elif "Portugués" in idioma:
            respuesta_al_usuario = "Desculpe, não posso ajudá-lo com isso. Hakuna Matata!"
        else:
            respuesta_al_usuario = "Lo siento, no puedo ayudarte con eso. Hakuna Matata!"
    
    return respuesta_al_usuario

In [81]:
print(history_answer(pregunta1))
print(history_answer(pregunta2))
print(history_answer(pregunta3))
# print(history_answer(pregunta4))
print(history_answer(pregunta5))
print(history_answer(pregunta2))


Thomas regresó al feudo cambiado, con la mirada fija en el horizonte, marcado por la muerte y la violencia que había presenciado en la batalla, dejando atrás su inocencia enterrada en el campo de batalla 🏰⚔️😔. Hakuna Matata!
Thomas returned from the battle a changed man, his innocence lost and replaced by a haunting gaze that spoke of death and violence, leaving his mother in tears as she realized her son was no longer the same. 😢💔 Hakuna Matata!
Thomas mudou profundamente depois da batalha, tornando-se um jovem marcado pela morte e violência, com um olhar fixo no horizonte e a inocência enterrada no campo de batalha 🏰⚔️😔. Hakuna Matata!
Sorry, I can't help you with that. Hakuna Matata!
Thomas returned from the battle a changed man, his innocence lost and replaced by a haunting gaze that spoke of death and violence, leaving his mother in tears as she realized her son was no longer the same. 😢💔 Hakuna Matata!


In [80]:
cache_respuestas = {}  # Reiniciar caché
print(cache_respuestas)

{}


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [3]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

# chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    responses = {
        "hola": "¡Hola! ¿En qué puedo ayudarte?",
        "adiós": "¡Hasta luego!",
    }

    return responses.get(message.lower(), "Lo siento, no entiendo esa pregunta.")

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [ ]:
# print(conversation_history)

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.